# xgboost model for homeless rate prediction

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import matplotlib.pyplot as plt

In [ ]:
# Load the data VERSION 2
train_data = pd.read_csv("train.csv")

# Separate features and target
X = train_data.drop(['ID', 'HOMELESS_RATE'], axis=1)
y = train_data['HOMELESS_RATE']

print("Data shape:", X.shape)
print("\nTarget statistics:")
print(f"  Mean: {y.mean():.6f}")
print(f"  Std: {y.std():.6f}")
print(f"  Min: {y.min():.6f}")
print(f"  Max: {y.max():.6f}")
print(f"  Median: {y.median():.6f}")

In [ ]:
# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nTraining set size:", X_train.shape)
print("Validation set size:", X_val.shape)

In [ ]:
# XGBoost Model (Simple approach)
print("\n\nTraining XGBoost Model...")

xgb_model = xgb.XGBRegressor(n_estimators=50, random_state=42)

# Cross Validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)
xgb_cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=cv, scoring="neg_mean_squared_error")
xgb_cv_mse = -xgb_cv_scores
print(f"\nXGBoost Cross-validation:")
print(f"  CV MSE scores: {xgb_cv_mse}")
print(f"  Average CV MSE: {np.mean(xgb_cv_mse):.10f}")
print(f"  Average CV RMSE: {np.sqrt(np.mean(xgb_cv_mse)):.10f}")

In [ ]:
# Fit on training set
xgb_model.fit(X_train, y_train)
xgb_train_pred = xgb_model.predict(X_train)
xgb_val_pred = xgb_model.predict(X_val)

# Evaluation
xgb_train_mse = mean_squared_error(y_train, xgb_train_pred)
xgb_val_mse = mean_squared_error(y_val, xgb_val_pred)

print("\nXGBoost Performance:")
print("Training Metrics:")
print(f"  MSE:  {xgb_train_mse:.10f}")
print(f"  RMSE: {np.sqrt(xgb_train_mse):.10f}")
print(f"  MAE:  {mean_absolute_error(y_train, xgb_train_pred):.10f}")
print(f"  R2:   {r2_score(y_train, xgb_train_pred):.6f}")

print("\nValidation Metrics:")
print(f"  MSE:  {xgb_val_mse:.10f}")
print(f"  RMSE: {np.sqrt(xgb_val_mse):.10f}")
print(f"  MAE:  {mean_absolute_error(y_val, xgb_val_pred):.10f}")
print(f"  R2:   {r2_score(y_val, xgb_val_pred):.6f}")

In [ ]:
# Feature Importance Analysis
print("\n\nTop 15 Most Important Features:")

# XGBoost Feature Importance
xgb_feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'XGB_Importance': xgb_model.feature_importances_
}).sort_values('XGB_Importance', ascending=False)

print("\nXGBoost Top 15:")
print(xgb_feature_importance.head(15).to_string(index=False))

In [ ]:
# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Target distribution
axes[0, 0].hist(y, bins=50, edgecolor='black')
axes[0, 0].set_xlabel('Homeless Rate')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Target Distribution')
axes[0, 0].grid(True, alpha=0.3)

# 2. Predictions vs Actual - XGBoost
axes[0, 1].scatter(y_val, xgb_val_pred, alpha=0.5, s=30)
axes[0, 1].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
axes[0, 1].set_xlabel('Actual Homeless Rate')
axes[0, 1].set_ylabel('Predicted Homeless Rate')
axes[0, 1].set_title(f'XGBoost: Predictions vs Actual\nMSE = {xgb_val_mse:.8f}')
axes[0, 1].grid(True, alpha=0.3)

# 3. Residuals - XGBoost
xgb_residuals = y_val - xgb_val_pred
axes[1, 0].scatter(xgb_val_pred, xgb_residuals, alpha=0.5, s=30)
axes[1, 0].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1, 0].set_xlabel('Predicted Homeless Rate')
axes[1, 0].set_ylabel('Residuals')
axes[1, 0].set_title('XGBoost: Residual Plot')
axes[1, 0].grid(True, alpha=0.3)

# 4. Feature Importance
top_features = xgb_feature_importance.head(10)
axes[1, 1].barh(range(len(top_features)), top_features['XGB_Importance'])
axes[1, 1].set_yticks(range(len(top_features)))
axes[1, 1].set_yticklabels(top_features['Feature'], fontsize=8)
axes[1, 1].invert_yaxis()
axes[1, 1].set_xlabel('Importance')
axes[1, 1].set_title('XGBoost: Top 10 Features')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('xgb_model_analysis.png', dpi=300, bbox_inches='tight')
print("\nVisualization saved as 'xgb_model_analysis.png'")
plt.show()

In [ ]:
# Retrain on full training data for final predictions
print("\n\nRetraining models on full training data...")
final_xgb_model = xgb.XGBRegressor(n_estimators=50, random_state=42)
final_xgb_model.fit(X, y)

In [ ]:
# Generate predictions for test set
print("\nGenerating Test Set Predictions...")

try:
    test_data = pd.read_csv("test.csv")
    test_ids = test_data['ID']
    X_test = test_data.drop(['ID'], axis=1)
    
    # Generate predictions with both models
    xgb_test_pred = final_xgb_model.predict(X_test)
    
    # Ensure no negative predictions
    xgb_test_pred = np.maximum(xgb_test_pred, 0)
    
    # Create submission files
    xgb_submission = pd.DataFrame({
        'ID': test_ids,
        'HOMELESS_RATE': xgb_test_pred
    })
    xgb_submission.to_csv('xgb_submission.csv', index=False)
    print("XGBoost predictions saved to 'xgb_submission.csv'")
    
    print("\nTest prediction statistics:")
    print(f"XGBoost - Min: {xgb_test_pred.min():.8f}, Max: {xgb_test_pred.max():.8f}, Mean: {xgb_test_pred.mean():.8f}")
    
except FileNotFoundError:
    print("Test file not found. Skipping test predictions.")

print("\nModel training complete.")